# Tutorial: NVIDIA Canary-Qwen Inference Pipeline

This notebook demonstrates how to run the NVIDIA Canary-Qwen inference pipeline on a single sample file.

**What this notebook does:**
- Uses production orchestration (Azure blob storage)
- Runs the smallest model (`nvidia/canary-1b`)
- Processes just 1 sample file
- Shows outputs (inference results, hypothesis text)

**Note**: Canary-Qwen is a speech language model (SALM) that combines ASR with language understanding.

## 1. Setup: Import and Configure Parameters

In [ ]:
import sys
import os
from pathlib import Path
import pandas as pd
import torch
import gc

# Disable wandb for notebook runs (tutorials don't need tracking)
os.environ['WANDB_MODE'] = 'disabled'

# Set HuggingFace cache to use local models directory (avoid re-downloading)
os.environ['HF_HOME'] = str(Path.cwd().parent / "models/canary")
os.environ['TRANSFORMERS_CACHE'] = str(Path.cwd().parent / "models/canary")

# Add scripts directory to path
sys.path.insert(0, str(Path.cwd().parent / "scripts"))

from infer_canary import run

# System and GPU Memory Check
print("=" * 70)
print("Memory Status Check")
print("=" * 70)

# Check system RAM
try:
    import psutil
    mem = psutil.virtual_memory()
    ram_used = mem.used / 1024**3
    ram_total = mem.total / 1024**3
    ram_available = mem.available / 1024**3
    
    print(f"System RAM: {ram_used:.1f}/{ram_total:.1f} GB used")
    print(f"Available: {ram_available:.1f} GB")
    
    if ram_available < 10:
        print(f"⚠️  WARNING: Low RAM! Canary needs ~10 GB free for model loading")
        print(f"   Current available: {ram_available:.1f} GB")
except ImportError:
    print("psutil not installed, skipping RAM check")

# Check GPU memory
if torch.cuda.is_available():
    print()
    for i in range(torch.cuda.device_count()):
        total = torch.cuda.get_device_properties(i).total_memory / 1024**3
        allocated = torch.cuda.memory_allocated(i) / 1024**3
        reserved = torch.cuda.memory_reserved(i) / 1024**3
        free = total - reserved
        
        print(f"GPU {i}: {torch.cuda.get_device_name(i)}")
        print(f"  Total: {total:.2f} GB")
        print(f"  Allocated: {allocated:.2f} GB")
        print(f"  Reserved: {reserved:.2f} GB")
        print(f"  Free: {free:.2f} GB")
        
        if free < 3.0:
            print(f"  ⚠️  WARNING: Low GPU memory! Only {free:.2f} GB free")
else:
    print("\n⚠️  No CUDA GPU available - Canary requires GPU!")

print("=" * 70)
print("\n💡 Note: Wandb logging is disabled for tutorial runs")

## 1.1 GPU Memory Cleanup (Run if needed)

If you see "CUDA out of memory" errors, run this cell to clear GPU memory from previous runs:

In [ ]:
# Clean up GPU memory
import torch
import gc

# Clear PyTorch CUDA cache
torch.cuda.empty_cache()

# Force Python garbage collection
gc.collect()

print("✓ GPU memory cleared")

# Show updated memory status
if torch.cuda.is_available():
    for i in range(torch.cuda.device_count()):
        allocated = torch.cuda.memory_allocated(i) / 1024**3
        reserved = torch.cuda.memory_reserved(i) / 1024**3
        total = torch.cuda.get_device_properties(i).total_memory / 1024**3
        free = total - reserved
        print(f"GPU {i}: {allocated:.2f} GB allocated, {reserved:.2f} GB reserved, {free:.2f} GB free")

## 2. Configure Inference Parameters

Set parameters directly in Python (no yaml file needed):

In [ ]:
# Build configuration dictionary
cfg = {
    "experiment_id": "tutorial-canary-sample1",
    
    # Model settings
    "model": {
        "name": "canary",
        "path": "nvidia/canary-qwen-2.5b",  # Options: nvidia/canary-1b-v2, nvidia/canary-1b-flash, nvidia/canary-qwen-2.5b
        "device": "cuda",  # Canary requires GPU (use "cuda")
        "prompt": "Transcribe the following:",  # User prompt for the model
        "max_new_tokens": 512,  # Max tokens to generate
        "batch_size": 1  # Process one file at a time
    },
    
    # Input settings
    "input": {
        "source": "azure_blob",
        "parquet_path": "../data/raw/loc/veterans_history_project_resources_pre2010.parquet",  # Relative to notebook dir
        "blob_prefix": "loc_vhp",
        "sample_size": 1,  # Just 1 file for tutorial
        "duration_sec": 300,  # First 5 minutes only (faster)
        "sample_rate": 16000  # Canary expects 16kHz
    },
    
    # Output settings
    "output": {
        "dir": "../outputs/tutorial-canary-sample1",  # Relative to notebook dir
        "save_per_file": True  # Save individual hypothesis files
    },
    
    # Evaluation (optional)
    "evaluation": {
        "use_whisper_normalizer": True
    }
}

print("Configuration:")
print(f"  Model: {cfg['model']['path']} (~2.5GB)")
print(f"  Device: {cfg['model']['device']} (GPU required)")
print(f"  Sample size: {cfg['input']['sample_size']} file(s)")
print(f"  Duration: {cfg['input']['duration_sec']}s (first 5 minutes)")
print(f"  Output: {cfg['output']['dir']}")
print(f"\n⚠️  Note: Canary requires a CUDA-enabled GPU")
print(f"\n💡 Tip: First run will download the model (~2.5GB). Subsequent runs use cached model.")

## 3. Run Inference

This will:
1. Load 1 sample from the parquet file
2. Download audio from Azure blob storage
3. Preprocess audio (trim to 300s, convert to 16kHz mono WAV)
4. Load Canary-Qwen model
5. Run transcription with speech-language model
6. Save results

**Note**: First run will download the model (~1GB for canary-1b)

In [ ]:
# Run the inference pipeline
run(cfg)

## 4. View Results

### 4.1 Inference Results (Parquet)

In [ ]:
# Load results parquet
results_path = Path(cfg['output']['dir']) / "inference_results.parquet"
df_results = pd.read_parquet(results_path)

print(f"Results shape: {df_results.shape}")
print(f"\nColumns: {list(df_results.columns)}")
print(f"\nFirst row:")
df_results.head()

### 4.2 Hypothesis Text

In [ ]:
# Display the transcription
row = df_results.iloc[0]

print("=" * 70)
print(f"File ID: {row['file_id']}")
print(f"Collection: {row['collection_number']}")
print(f"Duration: {row['duration_sec']:.1f}s")
print(f"Processing time: {row['processing_time_sec']:.1f}s")
print(f"Status: {row['status']}")
print("=" * 70)
print(f"\nTranscription (hypothesis):\n")
print(row['hypothesis'][:500] + "..." if len(row['hypothesis']) > 500 else row['hypothesis'])

### 4.3 Individual Hypothesis File

In [ ]:
# Check individual hypothesis file
hyp_file = Path(cfg['output']['dir']) / f"hyp_{row['file_id']}.txt"

if hyp_file.exists():
    print(f"Hypothesis file: {hyp_file}")
    print(f"\nContent:\n")
    print(hyp_file.read_text()[:500] + "..." if len(hyp_file.read_text()) > 500 else hyp_file.read_text())
else:
    print(f"Hypothesis file not found: {hyp_file}")

## 5. Understanding the Output

**Key output files:**
- `inference_results.parquet`: All results in structured format (hypothesis, duration, status, etc.)
- `hyp_{file_id}.txt`: Individual hypothesis text files (one per audio file)
- `hyp_canary.txt`: Combined hypothesis text (all transcriptions concatenated)
- `canary_log_*.txt`: Detailed inference log

**Key result columns:**
- `file_id`: Unique identifier
- `collection_number`: VHP collection number
- `hypothesis`: Model transcription output
- `ground_truth`: Reference transcript (for evaluation)
- `duration_sec`: Audio duration processed
- `processing_time_sec`: Time taken for inference
- `status`: success/error
- `model_name`: Model identifier

**Available Canary models:**
- `nvidia/canary-1b-v2`: ~1GB, multilingual (25 languages), good accuracy
- `nvidia/canary-1b-flash`: ~883M params, fastest (1000+ RTFx), 4 languages
- `nvidia/canary-qwen-2.5b`: ~2.5GB, best accuracy, English with punctuation (recommended for benchmarking)

## 6. Understanding Canary-Qwen

Canary-Qwen is a **Speech Language Model (SALM)** that combines:
- **ASR (Automatic Speech Recognition)**: Transcribes audio to text
- **Language understanding**: Can follow instructions and provide structured outputs

**Key features:**
- Supports multiple languages
- Can be prompted with instructions (e.g., "Transcribe the following:", "Summarize this audio:")
- Better at handling conversational speech
- Requires GPU (CUDA)

**Performance** (from Artificial Analysis benchmark):
- Canary performs competitively with Whisper and other open-source models
- Better at preserving speaker turns and conversational structure

## 7. Next Steps

To run on more files:
1. Increase `sample_size` (e.g., 10, 50, 100)
2. Set `duration_sec: null` for full audio
3. Try the larger model: `"nvidia/canary-qwen-2.5b"` for better accuracy
4. Customize the prompt: `"prompt": "Transcribe this interview:"`
5. Adjust `max_new_tokens` for longer transcriptions (default: 512)

To evaluate results:
```python
from scripts.evaluate import evaluate_results
metrics = evaluate_results(df_results, use_whisper_normalizer=True)
print(f"WER: {metrics['wer']:.2%}")
```

**Recommended config for benchmarking:**
```python
cfg = {
    "model": {
        "path": "nvidia/canary-qwen-2.5b",  # Better accuracy
        "device": "cuda",
        "max_new_tokens": 1024  # Longer transcriptions
    },
    "input": {
        "sample_size": 500,  # Full benchmark
        "duration_sec": null  # Full audio
    }
}
```